## Usage Checker

This notebook demonstrates how to check the usage of node definitions for node libraries.

The utility scans one or more libraries
    * Each definition is examined to see if it is implemented as a node graph. 
        * If it is the list of nodes it uses is returned.
        * For each node that is used a list of node definitions that use it is returned.
    * A list of definitions which are not implemented as graphs as well as a list of nodes that are missing implementations are returned.

The utility also provides usage rate for node definitions and a hint of complexity of a node definition which uses other nodes by providing a count of
the number of unique nodes it uses.

In [1]:
import MaterialX as mx
import mtlxutils.mxfile as mxf
from IPython.display import display_markdown

stdlib = mx.createDocument()
searchPath = mx.getDefaultDataSearchPath()
libraryFolders = mx.getDefaultDataLibraryFolders() 
try:
    libFiles = mx.loadLibraries(libraryFolders, searchPath, stdlib)
    print('Loaded %s standard library definitions for version %s' % (len(stdlib.getNodeDefs()), mx.getVersionString()))
except mx.Exception as err:
    print('Failed to load standard library definitions: "', err, '"')

doc = mx.createDocument()
doc.importLibrary(stdlib)


Loaded 780 standard library definitions for version 1.39.2


## Utility Methods

In [2]:

def getNodeDefUsage(doc, libraryFilter=None):
    
    # List of nodes used by another node
    nodes_use = dict()
    nodes_used_by = dict()
    nongraph_nodes = []
    unimplemented = []

    for nodedef in doc.getNodeDefs():
        found = True
        if libraryFilter:
            found = False
            for lib in libraryFilter:
                if lib in nodedef.getSourceUri():
                    found = True
                    break
        if not found:
            continue

        nodename = nodedef.getNodeString()
        impl = nodedef.getImplementation()
        if impl:
            nodes_use_list = [] 
            for node in impl.getChildrenOfType(mx.Node):
                #print('scan:', node.getName())
                nd = node.getNodeDef()
                if nd:
                    ns = nd.getNodeString()
                    if ns not in nodes_use_list:                    
                        nodes_use_list.append(ns)

                    # Look for ns key in nodes_used_by dictionary
                    if ns not in nodes_used_by:
                        nodes_used_by[ns] = []

                    if nodename not in nodes_used_by[ns]:
                        nodes_used_by[ns].append(nodename)
                else:
                    print('Skip unknown node type: ', node.getName())

            if len(nodes_use_list) > 0:
                nodes_use[nodename] = nodes_use_list
            else:
                if nodename not in nongraph_nodes:
                    nongraph_nodes.append(nodename)
        else:
            if nodename not in unimplemented:
                unimplemented.append(nodename)

    # Sort 
    nongraph_nodes = sorted(nongraph_nodes)
    unimplemented = sorted(unimplemented)
    nodes_use = {k: v for k, v in sorted(nodes_use.items(), key=lambda item: len(item[1]), reverse=True)}
    nodes_used_by = {k: v for k, v in sorted(nodes_used_by.items(), key=lambda item: len(item[1]), reverse=True)}

    return nodes_use, nodes_used_by, nongraph_nodes, unimplemented

def printNodeDefUsage(nodes_use, nodes_used_by, nongraph_nodes, unimplemented):
    #
    result = '| Node Definition | Uses |\n| --- | --- |'
    for nu in nodes_use:
        if (len(nodes_use[nu]) > 0):
            result += '\n| %s | %d nodes: %s |' % (nu, len(nodes_use[nu]), ', '.join(sorted(nodes_use[nu])))
    display_markdown(result, raw=True)

    result = '| Node Definition | Used by |\n| --- | --- |'
    for nu in nodes_used_by:
        if (len(nodes_used_by[nu]) > 0):
            result += '\n| %s | %d nodes: %s |' % (nu, len(nodes_used_by[nu]), ', '.join(sorted(nodes_used_by[nu])) )
    display_markdown(result, raw=True)

    if len(nongraph_nodes) > 0:
        result = '| Non-Graph Nodes: %d |\n| --- |\n' % (len(nongraph_nodes))
        result += '|' + ', '.join(nongraph_nodes) + '|'
        display_markdown(result, raw=True)

    if (len(unimplemented) > 0):
        result = '| Unimplemented Nodes %d |\n| --- |\n' % (len(unimplemented))
        #for nu in unimplemented:
        #    result += '\n| %s |' % (nu)
        result += '|' + ', '.join(unimplemented) + '|'
        display_markdown(result, raw=True)
    

## Node Usages Results for "Standard Library"

In [3]:
nodes_use, nodes_used_by, nongraph_nodes, unimplemented = getNodeDefUsage(doc, ['stdlib', 'nprlib'])

printNodeDefUsage(nodes_use, nodes_used_by, nongraph_nodes, unimplemented)

| Node Definition | Uses |
| --- | --- |
| triplanarprojection | 13 nodes: absval, add, clamp, combine2, divide, dotproduct, extract, image, multiply, normalize, power, separate3, switch |
| hexagon | 13 nodes: absval, clamp, combine2, constant, dotproduct, extract, ifgreater, min, multiply, separate2, separate3, sqrt, subtract |
| unifiednoise2d | 12 nodes: add, cellnoise2d, combine3, fractal3d, multiply, noise2d, range, rotate2d, separate2, subtract, switch, worleynoise2d |
| tiledcircles | 12 nodes: add, circle, combine2, convert, divide, ifequal, ifgreater, max, modulo, multiply, separate2, subtract |
| tiledcloverleafs | 12 nodes: add, cloverleaf, combine2, convert, divide, ifequal, ifgreater, max, modulo, multiply, separate2, subtract |
| tiledhexagons | 12 nodes: add, combine2, convert, divide, hexagon, ifequal, ifgreater, max, modulo, multiply, separate2, subtract |
| gooch_shade | 11 nodes: add, divide, dotproduct, max, mix, multiply, normal, normalize, power, reflect, viewdirection |
| ramp | 11 nodes: absval, add, atan2, divide, ifgreater, magnitude, multiply, ramp_gradient, separate2, subtract, switch |
| crosshatch | 11 nodes: add, combine2, convert, ifequal, ifgreater, line, max, modulo, multiply, separate2, subtract |
| unifiednoise3d | 10 nodes: add, cellnoise3d, fractal3d, multiply, noise3d, range, rotate3d, subtract, switch, worleynoise3d |
| grid | 10 nodes: absval, add, convert, ifequal, ifgreater, min, modulo, multiply, separate2, subtract |
| range | 8 nodes: absval, clamp, divide, ifequal, multiply, power, remap, sign |
| ramp_gradient | 7 nodes: clamp, ifgreater, ifgreatereq, mix, remap, smoothstep, switch |
| line | 7 nodes: clamp, distance, divide, dotproduct, ifgreater, multiply, subtract |
| checkerboard | 6 nodes: dotproduct, floor, mix, modulo, multiply, subtract |
| cloverleaf | 6 nodes: add, circle, combine2, max, separate2, subtract |
| refract | 6 nodes: add, dotproduct, ifgreater, multiply, sqrt, subtract |
| facingratio | 5 nodes: absval, dotproduct, ifequal, invert, multiply |
| place2d | 5 nodes: add, divide, rotate2d, subtract, switch |
| hsvadjust | 5 nodes: add, convert, hsvtorgb, multiply, rgbtohsv |
| colorcorrect | 5 nodes: colorcorrect, combine3, combine4, separate3, separate4 |
| tiledimage | 4 nodes: divide, image, multiply, subtract |
| randomfloat | 4 nodes: cellnoise2d, combine2, convert, range |
| circle | 4 nodes: dotproduct, ifgreater, multiply, subtract |
| safepower | 4 nodes: absval, multiply, power, sign |
| ramp4 | 3 nodes: clamp, extract, mix |
| trianglewave | 3 nodes: absval, modulo, subtract |
| reflect | 3 nodes: dotproduct, multiply, subtract |
| smoothstep | 3 nodes: combine4, separate4, smoothstep |
| contrast | 3 nodes: add, multiply, subtract |
| overlay | 3 nodes: combine4, overlay, separate4 |
| xor | 3 nodes: and, not, or |
| noise2d | 2 nodes: convert, noise2d |
| noise3d | 2 nodes: convert, noise3d |
| fractal3d | 2 nodes: convert, fractal3d |
| randomcolor | 2 nodes: convert, randomcolor |
| bump | 2 nodes: heighttonormal, normalmap |
| distance | 2 nodes: magnitude, subtract |
| saturate | 2 nodes: luminance, mix |
| convert | 2 nodes: convert, surface_unlit |
| switch | 1 nodes: ifgreater |
| separate2 | 1 nodes: extract |
| separate3 | 1 nodes: extract |
| separate4 | 1 nodes: extract |

| Node Definition | Used by |
| --- | --- |
| multiply | 25 nodes: checkerboard, circle, colorcorrect, contrast, crosshatch, facingratio, gooch_shade, grid, hexagon, hsvadjust, line, overlay, ramp, randomfloat, range, reflect, refract, safepower, tiledcircles, tiledcloverleafs, tiledhexagons, tiledimage, triplanarprojection, unifiednoise2d, unifiednoise3d |
| subtract | 22 nodes: checkerboard, circle, cloverleaf, colorcorrect, contrast, crosshatch, distance, grid, hexagon, line, overlay, place2d, ramp, reflect, refract, tiledcircles, tiledcloverleafs, tiledhexagons, tiledimage, trianglewave, unifiednoise2d, unifiednoise3d |
| add | 17 nodes: cloverleaf, colorcorrect, contrast, crosshatch, gooch_shade, grid, hsvadjust, place2d, ramp, randomcolor, refract, tiledcircles, tiledcloverleafs, tiledhexagons, triplanarprojection, unifiednoise2d, unifiednoise3d |
| ifgreater | 12 nodes: circle, crosshatch, grid, hexagon, line, ramp, ramp_gradient, refract, switch, tiledcircles, tiledcloverleafs, tiledhexagons |
| convert | 12 nodes: convert, crosshatch, fractal3d, grid, hsvadjust, noise2d, noise3d, randomcolor, randomfloat, tiledcircles, tiledcloverleafs, tiledhexagons |
| separate2 | 11 nodes: cloverleaf, convert, crosshatch, grid, hexagon, ramp, smoothstep, tiledcircles, tiledcloverleafs, tiledhexagons, unifiednoise2d |
| divide | 10 nodes: gooch_shade, line, place2d, ramp, range, tiledcircles, tiledcloverleafs, tiledhexagons, tiledimage, triplanarprojection |
| combine2 | 10 nodes: cloverleaf, convert, crosshatch, hexagon, randomfloat, smoothstep, tiledcircles, tiledcloverleafs, tiledhexagons, triplanarprojection |
| dotproduct | 9 nodes: checkerboard, circle, facingratio, gooch_shade, hexagon, line, reflect, refract, triplanarprojection |
| absval | 8 nodes: facingratio, grid, hexagon, ramp, range, safepower, trianglewave, triplanarprojection |
| ifequal | 8 nodes: convert, crosshatch, facingratio, grid, range, tiledcircles, tiledcloverleafs, tiledhexagons |
| extract | 7 nodes: convert, hexagon, ramp4, separate2, separate3, separate4, triplanarprojection |
| modulo | 7 nodes: checkerboard, crosshatch, grid, tiledcircles, tiledcloverleafs, tiledhexagons, trianglewave |
| mix | 6 nodes: checkerboard, gooch_shade, overlay, ramp4, ramp_gradient, saturate |
| max | 6 nodes: cloverleaf, crosshatch, gooch_shade, tiledcircles, tiledcloverleafs, tiledhexagons |
| switch | 6 nodes: place2d, ramp, ramp_gradient, triplanarprojection, unifiednoise2d, unifiednoise3d |
| clamp | 6 nodes: hexagon, line, ramp4, ramp_gradient, range, triplanarprojection |
| separate3 | 6 nodes: colorcorrect, convert, hexagon, overlay, smoothstep, triplanarprojection |
| combine3 | 6 nodes: colorcorrect, convert, overlay, randomcolor, smoothstep, unifiednoise2d |
| power | 5 nodes: colorcorrect, gooch_shade, range, safepower, triplanarprojection |
| range | 5 nodes: colorcorrect, randomcolor, randomfloat, unifiednoise2d, unifiednoise3d |
| separate4 | 4 nodes: colorcorrect, convert, overlay, smoothstep |
| combine4 | 4 nodes: colorcorrect, convert, overlay, smoothstep |
| fractal3d | 3 nodes: fractal3d, unifiednoise2d, unifiednoise3d |
| normalize | 2 nodes: gooch_shade, triplanarprojection |
| image | 2 nodes: tiledimage, triplanarprojection |
| magnitude | 2 nodes: distance, ramp |
| remap | 2 nodes: ramp_gradient, range |
| smoothstep | 2 nodes: ramp_gradient, smoothstep |
| ifgreatereq | 2 nodes: overlay, ramp_gradient |
| noise2d | 2 nodes: noise2d, unifiednoise2d |
| noise3d | 2 nodes: noise3d, unifiednoise3d |
| cellnoise2d | 2 nodes: randomfloat, unifiednoise2d |
| rotate2d | 2 nodes: place2d, unifiednoise2d |
| hsvtorgb | 2 nodes: hsvadjust, randomcolor |
| circle | 2 nodes: cloverleaf, tiledcircles |
| min | 2 nodes: grid, hexagon |
| sqrt | 2 nodes: hexagon, refract |
| sign | 2 nodes: range, safepower |
| not | 2 nodes: convert, xor |
| invert | 1 nodes: facingratio |
| normal | 1 nodes: gooch_shade |
| viewdirection | 1 nodes: gooch_shade |
| reflect | 1 nodes: gooch_shade |
| ramp_gradient | 1 nodes: ramp |
| atan2 | 1 nodes: ramp |
| worleynoise2d | 1 nodes: unifiednoise2d |
| cellnoise3d | 1 nodes: unifiednoise3d |
| worleynoise3d | 1 nodes: unifiednoise3d |
| rotate3d | 1 nodes: unifiednoise3d |
| ceil | 1 nodes: randomcolor |
| randomfloat | 1 nodes: randomcolor |
| randomcolor | 1 nodes: randomcolor |
| floor | 1 nodes: checkerboard |
| distance | 1 nodes: line |
| constant | 1 nodes: hexagon |
| line | 1 nodes: crosshatch |
| cloverleaf | 1 nodes: tiledcloverleafs |
| hexagon | 1 nodes: tiledhexagons |
| heighttonormal | 1 nodes: bump |
| normalmap | 1 nodes: bump |
| rgbtohsv | 1 nodes: hsvadjust |
| luminance | 1 nodes: saturate |
| contrast | 1 nodes: colorcorrect |
| saturate | 1 nodes: colorcorrect |
| hsvadjust | 1 nodes: colorcorrect |
| colorcorrect | 1 nodes: colorcorrect |
| overlay | 1 nodes: overlay |
| surface_unlit | 1 nodes: convert |
| and | 1 nodes: xor |
| or | 1 nodes: xor |

| Non-Graph Nodes: 101 |
| --- |
|absval, acos, add, and, asin, atan2, bitangent, blur, burn, ceil, cellnoise2d, cellnoise3d, clamp, combine2, combine3, combine4, constant, convert, cos, creatematrix, crossproduct, determinant, difference, disjointover, divide, dodge, dot, dotproduct, exp, extract, floor, fractal3d, frame, geomcolor, geompropvalue, geompropvalueuniform, heighttonormal, hsvtorgb, ifequal, ifgreater, ifgreatereq, image, in, inside, invert, invertmatrix, ln, luminance, magnitude, mask, matte, max, min, minus, mix, modulo, multiply, noise2d, noise3d, normal, normalize, normalmap, not, or, out, outside, over, plus, position, power, premult, ramplr, ramptb, remap, rgbtohsv, rotate2d, rotate3d, round, screen, sign, sin, smoothstep, splitlr, splittb, sqrt, subtract, surface_unlit, surfacematerial, tan, tangent, texcoord, time, transformmatrix, transformnormal, transformpoint, transformvector, transpose, unpremult, viewdirection, worleynoise2d, worleynoise3d|

| Unimplemented Nodes 1 |
| --- |
|volumematerial|

## Node Usage Results for "pbrlib, bxdf"

In [4]:
nodes_use, nodes_used_by, nongraph_nodes, unimplemented = getNodeDefUsage(doc, ['pbrlib', 'bxdf'])

printNodeDefUsage(nodes_use, nodes_used_by, nongraph_nodes, unimplemented)

| Node Definition | Uses |
| --- | --- |
| open_pbr_surface | 27 nodes: add, anisotropic_vdf, clamp, convert, dielectric_bsdf, divide, extract, generalized_schlick_bsdf, generalized_schlick_edf, ifgreater, layer, ln, max, min, mix, multiply, open_pbr_anisotropy, oren_nayar_diffuse_bsdf, power, sheen_bsdf, sign, sqrt, subsurface_bsdf, subtract, surface, translucent_bsdf, uniform_edf |
| standard_surface | 26 nodes: add, artistic_ior, clamp, conductor_bsdf, convert, dielectric_bsdf, divide, extract, generalized_schlick_edf, ifgreater, layer, luminance, max, mix, multiply, normalize, oren_nayar_diffuse_bsdf, power, rotate3d, roughness_anisotropy, sheen_bsdf, subsurface_bsdf, subtract, surface, translucent_bsdf, uniform_edf |
| gltf_pbr | 21 nodes: add, anisotropic_vdf, convert, dielectric_bsdf, divide, extract, generalized_schlick_bsdf, ifequal, ifgreatereq, layer, ln, max, min, mix, multiply, oren_nayar_diffuse_bsdf, roughness_anisotropy, sheen_bsdf, subtract, surface, uniform_edf |
| UsdPreviewSurface | 20 nodes: add, artistic_ior, conductor_bsdf, convert, dielectric_bsdf, divide, generalized_schlick_bsdf, ifequal, ifgreater, ifgreatereq, layer, mix, multiply, normalmap, oren_nayar_diffuse_bsdf, roughness_anisotropy, subtract, surface, switch, uniform_edf |
| LamaDielectric | 18 nodes: add, anisotropic_vdf, artistic_ior, clamp, combine2, convert, dielectric_bsdf, divide, extract, ifgreatereq, layer, max, multiply, normalize, power, rotate3d, subtract, switch |
| LamaGeneralizedSchlick | 15 nodes: add, anisotropic_vdf, combine2, convert, divide, generalized_schlick_bsdf, ifgreater, layer, max, multiply, normalize, power, rotate3d, subtract, switch |
| LamaConductor | 14 nodes: add, artistic_ior, clamp, combine2, conductor_bsdf, convert, ifgreatereq, max, multiply, normalize, power, rotate3d, subtract, switch |
| disney_principled | 13 nodes: burley_diffuse_bsdf, convert, dielectric_bsdf, generalized_schlick_bsdf, invert, layer, mix, multiply, roughness_anisotropy, roughness_dual, sheen_bsdf, subsurface_bsdf, surface |
| LamaIridescence | 13 nodes: add, clamp, combine2, dielectric_bsdf, ifgreatereq, max, mix, multiply, normal, normalize, power, rotate3d, subtract |
| open_pbr_surface_to_standard_surface | 10 nodes: add, constant, convert, divide, dot, ifgreater, mix, multiply, power, subtract |
| standard_surface_to_open_pbr_surface | 9 nodes: convert, dot, dotproduct, extract, ifequal, ifgreater, mix, multiply, power |
| standard_surface_to_gltf_pbr | 8 nodes: convert, divide, dot, dotproduct, ifequal, ifgreater, mix, multiply |
| standard_surface_to_UsdPreviewSurface | 7 nodes: convert, divide, dot, dotproduct, mix, multiply, subtract |
| open_pbr_anisotropy | 6 nodes: add, combine2, divide, invert, multiply, sqrt |
| gltf_colorimage | 5 nodes: combine3, dot, gltf_image, multiply, separate4 |
| gltf_normalmap | 5 nodes: divide, image, multiply, normalmap, place2d |
| UsdUVTexture | 5 nodes: add, convert, image, multiply, separate4 |
| gltf_image | 4 nodes: divide, image, multiply, place2d |
| LamaSheen | 4 nodes: add, multiply, power, sheen_bsdf |
| gltf_iridescence_thickness | 3 nodes: extract, gltf_image, mix |
| LamaDiffuse | 3 nodes: mix, multiply, oren_nayar_diffuse_bsdf |
| LamaAdd | 2 nodes: add, multiply |
| LamaLayer | 2 nodes: layer, multiply |
| LamaSSS | 2 nodes: multiply, subsurface_bsdf |
| LamaSurface | 2 nodes: surface, surfacematerial |
| glossiness_anisotropy | 2 nodes: invert, roughness_anisotropy |
| UsdPrimvarReader | 1 nodes: geompropvalue |
| UsdTransform2d | 1 nodes: place2d |
| LamaEmission | 1 nodes: uniform_edf |
| LamaMix | 1 nodes: mix |
| LamaTranslucent | 1 nodes: translucent_bsdf |

| Node Definition | Used by |
| --- | --- |
| multiply | 23 nodes: LamaAdd, LamaConductor, LamaDielectric, LamaDiffuse, LamaGeneralizedSchlick, LamaIridescence, LamaLayer, LamaSSS, LamaSheen, UsdPreviewSurface, UsdUVTexture, disney_principled, gltf_colorimage, gltf_image, gltf_normalmap, gltf_pbr, open_pbr_anisotropy, open_pbr_surface, open_pbr_surface_to_standard_surface, standard_surface, standard_surface_to_UsdPreviewSurface, standard_surface_to_gltf_pbr, standard_surface_to_open_pbr_surface |
| mix | 13 nodes: LamaDiffuse, LamaIridescence, LamaMix, UsdPreviewSurface, disney_principled, gltf_iridescence_thickness, gltf_pbr, open_pbr_surface, open_pbr_surface_to_standard_surface, standard_surface, standard_surface_to_UsdPreviewSurface, standard_surface_to_gltf_pbr, standard_surface_to_open_pbr_surface |
| convert | 13 nodes: LamaConductor, LamaDielectric, LamaGeneralizedSchlick, UsdPreviewSurface, UsdUVTexture, disney_principled, gltf_pbr, open_pbr_surface, open_pbr_surface_to_standard_surface, standard_surface, standard_surface_to_UsdPreviewSurface, standard_surface_to_gltf_pbr, standard_surface_to_open_pbr_surface |
| add | 13 nodes: LamaAdd, LamaConductor, LamaDielectric, LamaGeneralizedSchlick, LamaIridescence, LamaSheen, UsdPreviewSurface, UsdUVTexture, gltf_pbr, open_pbr_anisotropy, open_pbr_surface, open_pbr_surface_to_standard_surface, standard_surface |
| divide | 12 nodes: LamaDielectric, LamaGeneralizedSchlick, UsdPreviewSurface, gltf_image, gltf_normalmap, gltf_pbr, open_pbr_anisotropy, open_pbr_surface, open_pbr_surface_to_standard_surface, standard_surface, standard_surface_to_UsdPreviewSurface, standard_surface_to_gltf_pbr |
| subtract | 10 nodes: LamaConductor, LamaDielectric, LamaGeneralizedSchlick, LamaIridescence, UsdPreviewSurface, gltf_pbr, open_pbr_surface, open_pbr_surface_to_standard_surface, standard_surface, standard_surface_to_UsdPreviewSurface |
| power | 9 nodes: LamaConductor, LamaDielectric, LamaGeneralizedSchlick, LamaIridescence, LamaSheen, open_pbr_surface, open_pbr_surface_to_standard_surface, standard_surface, standard_surface_to_open_pbr_surface |
| layer | 8 nodes: LamaDielectric, LamaGeneralizedSchlick, LamaLayer, UsdPreviewSurface, disney_principled, gltf_pbr, open_pbr_surface, standard_surface |
| dielectric_bsdf | 7 nodes: LamaDielectric, LamaIridescence, UsdPreviewSurface, disney_principled, gltf_pbr, open_pbr_surface, standard_surface |
| max | 7 nodes: LamaConductor, LamaDielectric, LamaGeneralizedSchlick, LamaIridescence, gltf_pbr, open_pbr_surface, standard_surface |
| ifgreater | 7 nodes: LamaGeneralizedSchlick, UsdPreviewSurface, open_pbr_surface, open_pbr_surface_to_standard_surface, standard_surface, standard_surface_to_gltf_pbr, standard_surface_to_open_pbr_surface |
| surface | 6 nodes: LamaSurface, UsdPreviewSurface, disney_principled, gltf_pbr, open_pbr_surface, standard_surface |
| extract | 6 nodes: LamaDielectric, gltf_iridescence_thickness, gltf_pbr, open_pbr_surface, standard_surface, standard_surface_to_open_pbr_surface |
| sheen_bsdf | 5 nodes: LamaSheen, disney_principled, gltf_pbr, open_pbr_surface, standard_surface |
| roughness_anisotropy | 5 nodes: UsdPreviewSurface, disney_principled, glossiness_anisotropy, gltf_pbr, standard_surface |
| generalized_schlick_bsdf | 5 nodes: LamaGeneralizedSchlick, UsdPreviewSurface, disney_principled, gltf_pbr, open_pbr_surface |
| oren_nayar_diffuse_bsdf | 5 nodes: LamaDiffuse, UsdPreviewSurface, gltf_pbr, open_pbr_surface, standard_surface |
| uniform_edf | 5 nodes: LamaEmission, UsdPreviewSurface, gltf_pbr, open_pbr_surface, standard_surface |
| ifgreatereq | 5 nodes: LamaConductor, LamaDielectric, LamaIridescence, UsdPreviewSurface, gltf_pbr |
| dot | 5 nodes: gltf_colorimage, open_pbr_surface_to_standard_surface, standard_surface_to_UsdPreviewSurface, standard_surface_to_gltf_pbr, standard_surface_to_open_pbr_surface |
| clamp | 5 nodes: LamaConductor, LamaDielectric, LamaIridescence, open_pbr_surface, standard_surface |
| combine2 | 5 nodes: LamaConductor, LamaDielectric, LamaGeneralizedSchlick, LamaIridescence, open_pbr_anisotropy |
| rotate3d | 5 nodes: LamaConductor, LamaDielectric, LamaGeneralizedSchlick, LamaIridescence, standard_surface |
| normalize | 5 nodes: LamaConductor, LamaDielectric, LamaGeneralizedSchlick, LamaIridescence, standard_surface |
| subsurface_bsdf | 4 nodes: LamaSSS, disney_principled, open_pbr_surface, standard_surface |
| anisotropic_vdf | 4 nodes: LamaDielectric, LamaGeneralizedSchlick, gltf_pbr, open_pbr_surface |
| ifequal | 4 nodes: UsdPreviewSurface, gltf_pbr, standard_surface_to_gltf_pbr, standard_surface_to_open_pbr_surface |
| artistic_ior | 4 nodes: LamaConductor, LamaDielectric, UsdPreviewSurface, standard_surface |
| switch | 4 nodes: LamaConductor, LamaDielectric, LamaGeneralizedSchlick, UsdPreviewSurface |
| invert | 3 nodes: disney_principled, glossiness_anisotropy, open_pbr_anisotropy |
| image | 3 nodes: UsdUVTexture, gltf_image, gltf_normalmap |
| place2d | 3 nodes: UsdTransform2d, gltf_image, gltf_normalmap |
| translucent_bsdf | 3 nodes: LamaTranslucent, open_pbr_surface, standard_surface |
| conductor_bsdf | 3 nodes: LamaConductor, UsdPreviewSurface, standard_surface |
| dotproduct | 3 nodes: standard_surface_to_UsdPreviewSurface, standard_surface_to_gltf_pbr, standard_surface_to_open_pbr_surface |
| ln | 2 nodes: gltf_pbr, open_pbr_surface |
| min | 2 nodes: gltf_pbr, open_pbr_surface |
| gltf_image | 2 nodes: gltf_colorimage, gltf_iridescence_thickness |
| separate4 | 2 nodes: UsdUVTexture, gltf_colorimage |
| normalmap | 2 nodes: UsdPreviewSurface, gltf_normalmap |
| sqrt | 2 nodes: open_pbr_anisotropy, open_pbr_surface |
| generalized_schlick_edf | 2 nodes: open_pbr_surface, standard_surface |
| burley_diffuse_bsdf | 1 nodes: disney_principled |
| roughness_dual | 1 nodes: disney_principled |
| combine3 | 1 nodes: gltf_colorimage |
| open_pbr_anisotropy | 1 nodes: open_pbr_surface |
| sign | 1 nodes: open_pbr_surface |
| luminance | 1 nodes: standard_surface |
| geompropvalue | 1 nodes: UsdPrimvarReader |
| geompropvalueuniform | 1 nodes: UsdPrimvarReader |
| normal | 1 nodes: LamaIridescence |
| surfacematerial | 1 nodes: LamaSurface |
| constant | 1 nodes: open_pbr_surface_to_standard_surface |

| Non-Graph Nodes: 30 |
| --- |
|absorption_vdf, add, anisotropic_vdf, artistic_ior, blackbody, burley_diffuse_bsdf, chiang_hair_absorption_from_color, chiang_hair_bsdf, chiang_hair_roughness, conductor_bsdf, conical_edf, deon_hair_absorption_from_melanin, dielectric_bsdf, displacement, generalized_schlick_bsdf, generalized_schlick_edf, layer, light, measured_edf, mix, multiply, oren_nayar_diffuse_bsdf, roughness_anisotropy, roughness_dual, sheen_bsdf, subsurface_bsdf, surface, translucent_bsdf, uniform_edf, volume|

## Node Usage Results for all Standard Library Nodes

In [5]:
nodes_use, nodes_used_by, nongraph_nodes, unimplemented = getNodeDefUsage(doc)

printNodeDefUsage(nodes_use, nodes_used_by, nongraph_nodes, unimplemented)

| Node Definition | Uses |
| --- | --- |
| open_pbr_surface | 27 nodes: add, anisotropic_vdf, clamp, convert, dielectric_bsdf, divide, extract, generalized_schlick_bsdf, generalized_schlick_edf, ifgreater, layer, ln, max, min, mix, multiply, open_pbr_anisotropy, oren_nayar_diffuse_bsdf, power, sheen_bsdf, sign, sqrt, subsurface_bsdf, subtract, surface, translucent_bsdf, uniform_edf |
| standard_surface | 26 nodes: add, artistic_ior, clamp, conductor_bsdf, convert, dielectric_bsdf, divide, extract, generalized_schlick_edf, ifgreater, layer, luminance, max, mix, multiply, normalize, oren_nayar_diffuse_bsdf, power, rotate3d, roughness_anisotropy, sheen_bsdf, subsurface_bsdf, subtract, surface, translucent_bsdf, uniform_edf |
| gltf_pbr | 21 nodes: add, anisotropic_vdf, convert, dielectric_bsdf, divide, extract, generalized_schlick_bsdf, ifequal, ifgreatereq, layer, ln, max, min, mix, multiply, oren_nayar_diffuse_bsdf, roughness_anisotropy, sheen_bsdf, subtract, surface, uniform_edf |
| UsdPreviewSurface | 20 nodes: add, artistic_ior, conductor_bsdf, convert, dielectric_bsdf, divide, generalized_schlick_bsdf, ifequal, ifgreater, ifgreatereq, layer, mix, multiply, normalmap, oren_nayar_diffuse_bsdf, roughness_anisotropy, subtract, surface, switch, uniform_edf |
| LamaDielectric | 18 nodes: add, anisotropic_vdf, artistic_ior, clamp, combine2, convert, dielectric_bsdf, divide, extract, ifgreatereq, layer, max, multiply, normalize, power, rotate3d, subtract, switch |
| LamaGeneralizedSchlick | 15 nodes: add, anisotropic_vdf, combine2, convert, divide, generalized_schlick_bsdf, ifgreater, layer, max, multiply, normalize, power, rotate3d, subtract, switch |
| LamaConductor | 14 nodes: add, artistic_ior, clamp, combine2, conductor_bsdf, convert, ifgreatereq, max, multiply, normalize, power, rotate3d, subtract, switch |
| disney_principled | 13 nodes: burley_diffuse_bsdf, convert, dielectric_bsdf, generalized_schlick_bsdf, invert, layer, mix, multiply, roughness_anisotropy, roughness_dual, sheen_bsdf, subsurface_bsdf, surface |
| LamaIridescence | 13 nodes: add, clamp, combine2, dielectric_bsdf, ifgreatereq, max, mix, multiply, normal, normalize, power, rotate3d, subtract |
| triplanarprojection | 13 nodes: absval, add, clamp, combine2, divide, dotproduct, extract, image, multiply, normalize, power, separate3, switch |
| hexagon | 13 nodes: absval, clamp, combine2, constant, dotproduct, extract, ifgreater, min, multiply, separate2, separate3, sqrt, subtract |
| unifiednoise2d | 12 nodes: add, cellnoise2d, combine3, fractal3d, multiply, noise2d, range, rotate2d, separate2, subtract, switch, worleynoise2d |
| tiledcircles | 12 nodes: add, circle, combine2, convert, divide, ifequal, ifgreater, max, modulo, multiply, separate2, subtract |
| tiledcloverleafs | 12 nodes: add, cloverleaf, combine2, convert, divide, ifequal, ifgreater, max, modulo, multiply, separate2, subtract |
| tiledhexagons | 12 nodes: add, combine2, convert, divide, hexagon, ifequal, ifgreater, max, modulo, multiply, separate2, subtract |
| gooch_shade | 11 nodes: add, divide, dotproduct, max, mix, multiply, normal, normalize, power, reflect, viewdirection |
| ramp | 11 nodes: absval, add, atan2, divide, ifgreater, magnitude, multiply, ramp_gradient, separate2, subtract, switch |
| crosshatch | 11 nodes: add, combine2, convert, ifequal, ifgreater, line, max, modulo, multiply, separate2, subtract |
| open_pbr_surface_to_standard_surface | 10 nodes: add, constant, convert, divide, dot, ifgreater, mix, multiply, power, subtract |
| unifiednoise3d | 10 nodes: add, cellnoise3d, fractal3d, multiply, noise3d, range, rotate3d, subtract, switch, worleynoise3d |
| grid | 10 nodes: absval, add, convert, ifequal, ifgreater, min, modulo, multiply, separate2, subtract |
| standard_surface_to_open_pbr_surface | 9 nodes: convert, dot, dotproduct, extract, ifequal, ifgreater, mix, multiply, power |
| standard_surface_to_gltf_pbr | 8 nodes: convert, divide, dot, dotproduct, ifequal, ifgreater, mix, multiply |
| range | 8 nodes: absval, clamp, divide, ifequal, multiply, power, remap, sign |
| standard_surface_to_UsdPreviewSurface | 7 nodes: convert, divide, dot, dotproduct, mix, multiply, subtract |
| ramp_gradient | 7 nodes: clamp, ifgreater, ifgreatereq, mix, remap, smoothstep, switch |
| line | 7 nodes: clamp, distance, divide, dotproduct, ifgreater, multiply, subtract |
| open_pbr_anisotropy | 6 nodes: add, combine2, divide, invert, multiply, sqrt |
| checkerboard | 6 nodes: dotproduct, floor, mix, modulo, multiply, subtract |
| cloverleaf | 6 nodes: add, circle, combine2, max, separate2, subtract |
| refract | 6 nodes: add, dotproduct, ifgreater, multiply, sqrt, subtract |
| gltf_colorimage | 5 nodes: combine3, dot, gltf_image, multiply, separate4 |
| gltf_normalmap | 5 nodes: divide, image, multiply, normalmap, place2d |
| UsdUVTexture | 5 nodes: add, convert, image, multiply, separate4 |
| facingratio | 5 nodes: absval, dotproduct, ifequal, invert, multiply |
| place2d | 5 nodes: add, divide, rotate2d, subtract, switch |
| hsvadjust | 5 nodes: add, convert, hsvtorgb, multiply, rgbtohsv |
| colorcorrect | 5 nodes: colorcorrect, combine3, combine4, separate3, separate4 |
| gltf_image | 4 nodes: divide, image, multiply, place2d |
| LamaSheen | 4 nodes: add, multiply, power, sheen_bsdf |
| g18_rec709_to_lin_rec709 | 4 nodes: combine2, convert, extract, g18_rec709_to_lin_rec709 |
| g22_rec709_to_lin_rec709 | 4 nodes: combine2, convert, extract, g22_rec709_to_lin_rec709 |
| rec709_display_to_lin_rec709 | 4 nodes: combine2, convert, extract, rec709_display_to_lin_rec709 |
| acescg_to_lin_rec709 | 4 nodes: acescg_to_lin_rec709, combine2, convert, extract |
| g22_ap1_to_lin_rec709 | 4 nodes: combine2, convert, extract, g22_ap1_to_lin_rec709 |
| srgb_texture_to_lin_rec709 | 4 nodes: combine2, convert, extract, srgb_texture_to_lin_rec709 |
| lin_adobergb_to_lin_rec709 | 4 nodes: combine2, convert, extract, lin_adobergb_to_lin_rec709 |
| adobergb_to_lin_rec709 | 4 nodes: adobergb_to_lin_rec709, combine2, convert, extract |
| srgb_displayp3_to_lin_rec709 | 4 nodes: combine2, convert, extract, srgb_displayp3_to_lin_rec709 |
| lin_displayp3_to_lin_rec709 | 4 nodes: combine2, convert, extract, lin_displayp3_to_lin_rec709 |
| tiledimage | 4 nodes: divide, image, multiply, subtract |
| randomfloat | 4 nodes: cellnoise2d, combine2, convert, range |
| circle | 4 nodes: dotproduct, ifgreater, multiply, subtract |
| safepower | 4 nodes: absval, multiply, power, sign |
| gltf_iridescence_thickness | 3 nodes: extract, gltf_image, mix |
| LamaDiffuse | 3 nodes: mix, multiply, oren_nayar_diffuse_bsdf |
| ramp4 | 3 nodes: clamp, extract, mix |
| trianglewave | 3 nodes: absval, modulo, subtract |
| reflect | 3 nodes: dotproduct, multiply, subtract |
| smoothstep | 3 nodes: combine4, separate4, smoothstep |
| contrast | 3 nodes: add, multiply, subtract |
| overlay | 3 nodes: combine4, overlay, separate4 |
| xor | 3 nodes: and, not, or |
| LamaAdd | 2 nodes: add, multiply |
| LamaLayer | 2 nodes: layer, multiply |
| LamaSSS | 2 nodes: multiply, subsurface_bsdf |
| LamaSurface | 2 nodes: surface, surfacematerial |
| glossiness_anisotropy | 2 nodes: invert, roughness_anisotropy |
| noise2d | 2 nodes: convert, noise2d |
| noise3d | 2 nodes: convert, noise3d |
| fractal3d | 2 nodes: convert, fractal3d |
| randomcolor | 2 nodes: convert, randomcolor |
| bump | 2 nodes: heighttonormal, normalmap |
| distance | 2 nodes: magnitude, subtract |
| saturate | 2 nodes: luminance, mix |
| convert | 2 nodes: convert, surface_unlit |
| UsdPrimvarReader | 1 nodes: geompropvalue |
| UsdTransform2d | 1 nodes: place2d |
| LamaEmission | 1 nodes: uniform_edf |
| LamaMix | 1 nodes: mix |
| LamaTranslucent | 1 nodes: translucent_bsdf |
| switch | 1 nodes: ifgreater |
| separate2 | 1 nodes: extract |
| separate3 | 1 nodes: extract |
| separate4 | 1 nodes: extract |

| Node Definition | Used by |
| --- | --- |
| multiply | 48 nodes: LamaAdd, LamaConductor, LamaDielectric, LamaDiffuse, LamaGeneralizedSchlick, LamaIridescence, LamaLayer, LamaSSS, LamaSheen, UsdPreviewSurface, UsdUVTexture, checkerboard, circle, colorcorrect, contrast, crosshatch, disney_principled, facingratio, gltf_colorimage, gltf_image, gltf_normalmap, gltf_pbr, gooch_shade, grid, hexagon, hsvadjust, line, open_pbr_anisotropy, open_pbr_surface, open_pbr_surface_to_standard_surface, overlay, ramp, randomfloat, range, reflect, refract, safepower, standard_surface, standard_surface_to_UsdPreviewSurface, standard_surface_to_gltf_pbr, standard_surface_to_open_pbr_surface, tiledcircles, tiledcloverleafs, tiledhexagons, tiledimage, triplanarprojection, unifiednoise2d, unifiednoise3d |
| convert | 35 nodes: LamaConductor, LamaDielectric, LamaGeneralizedSchlick, UsdPreviewSurface, UsdUVTexture, acescg_to_lin_rec709, adobergb_to_lin_rec709, convert, crosshatch, disney_principled, fractal3d, g18_rec709_to_lin_rec709, g22_ap1_to_lin_rec709, g22_rec709_to_lin_rec709, gltf_pbr, grid, hsvadjust, lin_adobergb_to_lin_rec709, lin_displayp3_to_lin_rec709, noise2d, noise3d, open_pbr_surface, open_pbr_surface_to_standard_surface, randomcolor, randomfloat, rec709_display_to_lin_rec709, srgb_displayp3_to_lin_rec709, srgb_texture_to_lin_rec709, standard_surface, standard_surface_to_UsdPreviewSurface, standard_surface_to_gltf_pbr, standard_surface_to_open_pbr_surface, tiledcircles, tiledcloverleafs, tiledhexagons |
| subtract | 32 nodes: LamaConductor, LamaDielectric, LamaGeneralizedSchlick, LamaIridescence, UsdPreviewSurface, checkerboard, circle, cloverleaf, colorcorrect, contrast, crosshatch, distance, gltf_pbr, grid, hexagon, line, open_pbr_surface, open_pbr_surface_to_standard_surface, overlay, place2d, ramp, reflect, refract, standard_surface, standard_surface_to_UsdPreviewSurface, tiledcircles, tiledcloverleafs, tiledhexagons, tiledimage, trianglewave, unifiednoise2d, unifiednoise3d |
| add | 31 nodes: LamaAdd, LamaConductor, LamaDielectric, LamaGeneralizedSchlick, LamaIridescence, LamaSheen, UsdPreviewSurface, UsdUVTexture, cloverleaf, colorcorrect, contrast, crosshatch, gltf_pbr, gooch_shade, grid, hsvadjust, open_pbr_anisotropy, open_pbr_surface, open_pbr_surface_to_standard_surface, place2d, ramp, randomcolor, refract, srgb_texture_to_lin_rec709, standard_surface, tiledcircles, tiledcloverleafs, tiledhexagons, triplanarprojection, unifiednoise2d, unifiednoise3d |
| combine2 | 25 nodes: LamaConductor, LamaDielectric, LamaGeneralizedSchlick, LamaIridescence, acescg_to_lin_rec709, adobergb_to_lin_rec709, cloverleaf, convert, crosshatch, g18_rec709_to_lin_rec709, g22_ap1_to_lin_rec709, g22_rec709_to_lin_rec709, hexagon, lin_adobergb_to_lin_rec709, lin_displayp3_to_lin_rec709, open_pbr_anisotropy, randomfloat, rec709_display_to_lin_rec709, smoothstep, srgb_displayp3_to_lin_rec709, srgb_texture_to_lin_rec709, tiledcircles, tiledcloverleafs, tiledhexagons, triplanarprojection |
| divide | 24 nodes: LamaDielectric, LamaGeneralizedSchlick, UsdPreviewSurface, adobergb_to_lin_rec709, gltf_image, gltf_normalmap, gltf_pbr, gooch_shade, line, open_pbr_anisotropy, open_pbr_surface, open_pbr_surface_to_standard_surface, place2d, ramp, range, srgb_texture_to_lin_rec709, standard_surface, standard_surface_to_UsdPreviewSurface, standard_surface_to_gltf_pbr, tiledcircles, tiledcloverleafs, tiledhexagons, tiledimage, triplanarprojection |
| extract | 23 nodes: LamaDielectric, acescg_to_lin_rec709, adobergb_to_lin_rec709, convert, g18_rec709_to_lin_rec709, g22_ap1_to_lin_rec709, g22_rec709_to_lin_rec709, gltf_iridescence_thickness, gltf_pbr, hexagon, lin_adobergb_to_lin_rec709, lin_displayp3_to_lin_rec709, open_pbr_surface, ramp4, rec709_display_to_lin_rec709, separate2, separate3, separate4, srgb_displayp3_to_lin_rec709, srgb_texture_to_lin_rec709, standard_surface, standard_surface_to_open_pbr_surface, triplanarprojection |
| mix | 20 nodes: LamaDiffuse, LamaIridescence, LamaMix, UsdPreviewSurface, checkerboard, disney_principled, gltf_iridescence_thickness, gltf_pbr, gooch_shade, open_pbr_surface, open_pbr_surface_to_standard_surface, overlay, ramp4, ramp_gradient, saturate, srgb_texture_to_lin_rec709, standard_surface, standard_surface_to_UsdPreviewSurface, standard_surface_to_gltf_pbr, standard_surface_to_open_pbr_surface |
| power | 20 nodes: LamaConductor, LamaDielectric, LamaGeneralizedSchlick, LamaIridescence, LamaSheen, adobergb_to_lin_rec709, colorcorrect, g18_rec709_to_lin_rec709, g22_ap1_to_lin_rec709, g22_rec709_to_lin_rec709, gooch_shade, open_pbr_surface, open_pbr_surface_to_standard_surface, range, rec709_display_to_lin_rec709, safepower, srgb_texture_to_lin_rec709, standard_surface, standard_surface_to_open_pbr_surface, triplanarprojection |
| ifgreater | 20 nodes: LamaGeneralizedSchlick, UsdPreviewSurface, circle, crosshatch, grid, hexagon, line, open_pbr_surface, open_pbr_surface_to_standard_surface, ramp, ramp_gradient, refract, srgb_texture_to_lin_rec709, standard_surface, standard_surface_to_gltf_pbr, standard_surface_to_open_pbr_surface, switch, tiledcircles, tiledcloverleafs, tiledhexagons |
| max | 19 nodes: LamaConductor, LamaDielectric, LamaGeneralizedSchlick, LamaIridescence, adobergb_to_lin_rec709, cloverleaf, crosshatch, g18_rec709_to_lin_rec709, g22_ap1_to_lin_rec709, g22_rec709_to_lin_rec709, gltf_pbr, gooch_shade, open_pbr_surface, rec709_display_to_lin_rec709, srgb_texture_to_lin_rec709, standard_surface, tiledcircles, tiledcloverleafs, tiledhexagons |
| ifequal | 12 nodes: UsdPreviewSurface, convert, crosshatch, facingratio, gltf_pbr, grid, range, standard_surface_to_gltf_pbr, standard_surface_to_open_pbr_surface, tiledcircles, tiledcloverleafs, tiledhexagons |
| dotproduct | 12 nodes: checkerboard, circle, facingratio, gooch_shade, hexagon, line, reflect, refract, standard_surface_to_UsdPreviewSurface, standard_surface_to_gltf_pbr, standard_surface_to_open_pbr_surface, triplanarprojection |
| clamp | 11 nodes: LamaConductor, LamaDielectric, LamaIridescence, hexagon, line, open_pbr_surface, ramp4, ramp_gradient, range, standard_surface, triplanarprojection |
| separate2 | 11 nodes: cloverleaf, convert, crosshatch, grid, hexagon, ramp, smoothstep, tiledcircles, tiledcloverleafs, tiledhexagons, unifiednoise2d |
| switch | 10 nodes: LamaConductor, LamaDielectric, LamaGeneralizedSchlick, UsdPreviewSurface, place2d, ramp, ramp_gradient, triplanarprojection, unifiednoise2d, unifiednoise3d |
| layer | 8 nodes: LamaDielectric, LamaGeneralizedSchlick, LamaLayer, UsdPreviewSurface, disney_principled, gltf_pbr, open_pbr_surface, standard_surface |
| combine3 | 8 nodes: colorcorrect, convert, gltf_colorimage, overlay, randomcolor, smoothstep, srgb_texture_to_lin_rec709, unifiednoise2d |
| absval | 8 nodes: facingratio, grid, hexagon, ramp, range, safepower, trianglewave, triplanarprojection |
| dielectric_bsdf | 7 nodes: LamaDielectric, LamaIridescence, UsdPreviewSurface, disney_principled, gltf_pbr, open_pbr_surface, standard_surface |
| ifgreatereq | 7 nodes: LamaConductor, LamaDielectric, LamaIridescence, UsdPreviewSurface, gltf_pbr, overlay, ramp_gradient |
| normalize | 7 nodes: LamaConductor, LamaDielectric, LamaGeneralizedSchlick, LamaIridescence, gooch_shade, standard_surface, triplanarprojection |
| constant | 7 nodes: acescg_to_lin_rec709, hexagon, lin_adobergb_to_lin_rec709, lin_displayp3_to_lin_rec709, open_pbr_surface_to_standard_surface, srgb_displayp3_to_lin_rec709, srgb_texture_to_lin_rec709 |
| separate3 | 7 nodes: colorcorrect, convert, hexagon, overlay, smoothstep, srgb_texture_to_lin_rec709, triplanarprojection |
| modulo | 7 nodes: checkerboard, crosshatch, grid, tiledcircles, tiledcloverleafs, tiledhexagons, trianglewave |
| surface | 6 nodes: LamaSurface, UsdPreviewSurface, disney_principled, gltf_pbr, open_pbr_surface, standard_surface |
| separate4 | 6 nodes: UsdUVTexture, colorcorrect, convert, gltf_colorimage, overlay, smoothstep |
| rotate3d | 6 nodes: LamaConductor, LamaDielectric, LamaGeneralizedSchlick, LamaIridescence, standard_surface, unifiednoise3d |
| sheen_bsdf | 5 nodes: LamaSheen, disney_principled, gltf_pbr, open_pbr_surface, standard_surface |
| roughness_anisotropy | 5 nodes: UsdPreviewSurface, disney_principled, glossiness_anisotropy, gltf_pbr, standard_surface |
| generalized_schlick_bsdf | 5 nodes: LamaGeneralizedSchlick, UsdPreviewSurface, disney_principled, gltf_pbr, open_pbr_surface |
| oren_nayar_diffuse_bsdf | 5 nodes: LamaDiffuse, UsdPreviewSurface, gltf_pbr, open_pbr_surface, standard_surface |
| uniform_edf | 5 nodes: LamaEmission, UsdPreviewSurface, gltf_pbr, open_pbr_surface, standard_surface |
| dot | 5 nodes: gltf_colorimage, open_pbr_surface_to_standard_surface, standard_surface_to_UsdPreviewSurface, standard_surface_to_gltf_pbr, standard_surface_to_open_pbr_surface |
| image | 5 nodes: UsdUVTexture, gltf_image, gltf_normalmap, tiledimage, triplanarprojection |
| range | 5 nodes: colorcorrect, randomcolor, randomfloat, unifiednoise2d, unifiednoise3d |
| invert | 4 nodes: disney_principled, facingratio, glossiness_anisotropy, open_pbr_anisotropy |
| subsurface_bsdf | 4 nodes: LamaSSS, disney_principled, open_pbr_surface, standard_surface |
| anisotropic_vdf | 4 nodes: LamaDielectric, LamaGeneralizedSchlick, gltf_pbr, open_pbr_surface |
| min | 4 nodes: gltf_pbr, grid, hexagon, open_pbr_surface |
| sqrt | 4 nodes: hexagon, open_pbr_anisotropy, open_pbr_surface, refract |
| artistic_ior | 4 nodes: LamaConductor, LamaDielectric, UsdPreviewSurface, standard_surface |
| transformmatrix | 4 nodes: acescg_to_lin_rec709, lin_adobergb_to_lin_rec709, lin_displayp3_to_lin_rec709, srgb_displayp3_to_lin_rec709 |
| combine4 | 4 nodes: colorcorrect, convert, overlay, smoothstep |
| place2d | 3 nodes: UsdTransform2d, gltf_image, gltf_normalmap |
| normalmap | 3 nodes: UsdPreviewSurface, bump, gltf_normalmap |
| translucent_bsdf | 3 nodes: LamaTranslucent, open_pbr_surface, standard_surface |
| sign | 3 nodes: open_pbr_surface, range, safepower |
| conductor_bsdf | 3 nodes: LamaConductor, UsdPreviewSurface, standard_surface |
| fractal3d | 3 nodes: fractal3d, unifiednoise2d, unifiednoise3d |
| ln | 2 nodes: gltf_pbr, open_pbr_surface |
| gltf_image | 2 nodes: gltf_colorimage, gltf_iridescence_thickness |
| generalized_schlick_edf | 2 nodes: open_pbr_surface, standard_surface |
| luminance | 2 nodes: saturate, standard_surface |
| normal | 2 nodes: LamaIridescence, gooch_shade |
| acescg_to_lin_rec709 | 2 nodes: acescg_to_lin_rec709, g22_ap1_to_lin_rec709 |
| srgb_texture_to_lin_rec709 | 2 nodes: srgb_displayp3_to_lin_rec709, srgb_texture_to_lin_rec709 |
| lin_adobergb_to_lin_rec709 | 2 nodes: adobergb_to_lin_rec709, lin_adobergb_to_lin_rec709 |
| magnitude | 2 nodes: distance, ramp |
| remap | 2 nodes: ramp_gradient, range |
| smoothstep | 2 nodes: ramp_gradient, smoothstep |
| noise2d | 2 nodes: noise2d, unifiednoise2d |
| noise3d | 2 nodes: noise3d, unifiednoise3d |
| cellnoise2d | 2 nodes: randomfloat, unifiednoise2d |
| rotate2d | 2 nodes: place2d, unifiednoise2d |
| hsvtorgb | 2 nodes: hsvadjust, randomcolor |
| circle | 2 nodes: cloverleaf, tiledcircles |
| not | 2 nodes: convert, xor |
| burley_diffuse_bsdf | 1 nodes: disney_principled |
| roughness_dual | 1 nodes: disney_principled |
| open_pbr_anisotropy | 1 nodes: open_pbr_surface |
| geompropvalue | 1 nodes: UsdPrimvarReader |
| geompropvalueuniform | 1 nodes: UsdPrimvarReader |
| surfacematerial | 1 nodes: LamaSurface |
| g18_rec709_to_lin_rec709 | 1 nodes: g18_rec709_to_lin_rec709 |
| g22_rec709_to_lin_rec709 | 1 nodes: g22_rec709_to_lin_rec709 |
| rec709_display_to_lin_rec709 | 1 nodes: rec709_display_to_lin_rec709 |
| g22_ap1_to_lin_rec709 | 1 nodes: g22_ap1_to_lin_rec709 |
| adobergb_to_lin_rec709 | 1 nodes: adobergb_to_lin_rec709 |
| srgb_displayp3_to_lin_rec709 | 1 nodes: srgb_displayp3_to_lin_rec709 |
| lin_displayp3_to_lin_rec709 | 1 nodes: lin_displayp3_to_lin_rec709 |
| viewdirection | 1 nodes: gooch_shade |
| reflect | 1 nodes: gooch_shade |
| ramp_gradient | 1 nodes: ramp |
| atan2 | 1 nodes: ramp |
| worleynoise2d | 1 nodes: unifiednoise2d |
| cellnoise3d | 1 nodes: unifiednoise3d |
| worleynoise3d | 1 nodes: unifiednoise3d |
| ceil | 1 nodes: randomcolor |
| randomfloat | 1 nodes: randomcolor |
| randomcolor | 1 nodes: randomcolor |
| floor | 1 nodes: checkerboard |
| distance | 1 nodes: line |
| line | 1 nodes: crosshatch |
| cloverleaf | 1 nodes: tiledcloverleafs |
| hexagon | 1 nodes: tiledhexagons |
| heighttonormal | 1 nodes: bump |
| rgbtohsv | 1 nodes: hsvadjust |
| contrast | 1 nodes: colorcorrect |
| saturate | 1 nodes: colorcorrect |
| hsvadjust | 1 nodes: colorcorrect |
| colorcorrect | 1 nodes: colorcorrect |
| overlay | 1 nodes: overlay |
| surface_unlit | 1 nodes: convert |
| and | 1 nodes: xor |
| or | 1 nodes: xor |

| Non-Graph Nodes: 131 |
| --- |
|absorption_vdf, absval, acos, add, and, anisotropic_vdf, artistic_ior, asin, atan2, bitangent, blackbody, blur, burley_diffuse_bsdf, burn, ceil, cellnoise2d, cellnoise3d, chiang_hair_absorption_from_color, chiang_hair_bsdf, chiang_hair_roughness, clamp, combine2, combine3, combine4, conductor_bsdf, conical_edf, constant, convert, cos, creatematrix, crossproduct, deon_hair_absorption_from_melanin, determinant, dielectric_bsdf, difference, directional_light, disjointover, displacement, divide, dodge, dot, dotproduct, exp, extract, floor, fractal3d, frame, generalized_schlick_bsdf, generalized_schlick_edf, geomcolor, geompropvalue, geompropvalueuniform, heighttonormal, hsvtorgb, ifequal, ifgreater, ifgreatereq, image, in, inside, invert, invertmatrix, layer, light, ln, luminance, magnitude, mask, matte, max, measured_edf, min, minus, mix, modulo, multiply, noise2d, noise3d, normal, normalize, normalmap, not, or, oren_nayar_diffuse_bsdf, out, outside, over, plus, point_light, position, power, premult, ramplr, ramptb, remap, rgbtohsv, rotate2d, rotate3d, roughness_anisotropy, roughness_dual, round, screen, sheen_bsdf, sign, sin, smoothstep, splitlr, splittb, spot_light, sqrt, subsurface_bsdf, subtract, surface, surface_unlit, surfacematerial, tan, tangent, texcoord, time, transformmatrix, transformnormal, transformpoint, transformvector, translucent_bsdf, transpose, uniform_edf, unpremult, viewdirection, volume, worleynoise2d, worleynoise3d|

| Unimplemented Nodes 1 |
| --- |
|volumematerial|